# Assignment

In this assignment we will train a convolutional neural network (CNN) to predict the imaging series protocol on prostate MRI. Identifying the correct series of interest if a key first step in most medical imaging problems. Instead of relying on manual DICOM header values which are prone to error, the trained CNN created as part of this exercise will ensure that any downstream algorithms can be executed with high accuracy.

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook (push to https://github.com/[username]/cs190/cnn/assignment.ipynb)
* final trained `*.hdf5` model file
* final compiled `*.csv` file with performance statistics

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

### Mount Google Drive

The Google Colab environment is transient and will reset after any prolonged break in activity. To retain important and/or large files between sessions, use the following lines of code to mount your personal Google drive to this Colab instance:

In [15]:
try:
    # --- Mount gdrive to /content/drive/My Drive/
    from google.colab import drive
    drive.mount('/content/drive')
    
except: pass

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Throughout this assignment we will use the following global `MOUNT_ROOT` variable to reference a location to store long-term data. If you are using a local Jupyter server and/or wish to store your data elsewhere, please update this variable now.

In [0]:
# --- Set data directory
MOUNT_ROOT = '/content/drive/My Drive'

### Select Tensorflow library version

This assignment will use the (new) Tensorflow 2.0 library. Use the following line of code to select this updated version:

In [0]:
# --- Select Tensorflow 2.0 (only in Google Colab)
% tensorflow_version 2.x

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [2]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

     |████████████████████████████████| 61kB 9.7MB/s 
     |████████████████████████████████| 276kB 41.9MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=520df6e8f0bc42de9b2b25a9de7b2588dc41b7672f1e6f0480a9b9161496ef0b
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Imports

Use the following lines to import any additional needed libraries:

In [0]:
import os, numpy as np, pandas as pd
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers
from jarvis.train import datasets

# Data

As in the tutorial, data for this assignment will consist of prostate MRI exams. Each image will consist of one of four different sequences (T2, low b-value DWI, high b-value DWI, ADC). In this initial exercise, the goal is to simply develop an algorith that is capable of differentiating image type so that downstream models for cancer prediction can be used properly. The following lines of code will:

1. Download the dataset (if not already present) 
2. Prepare the necessary Python generators to iterate through dataset
3. Prepare the corresponding Tensorflow Input(...) objects for model definition

In [4]:
# --- Download dataset
datasets.download(name='mr/prostatex')

# --- Prepare generators and model inputs
gen_train, gen_valid, client = datasets.prepare(name='mr/prostatex')
inputs = client.get_inputs(Input)

[ 2020-04-27 17:54:47 ] [====================] 100.000% : Extracting archive (0001377 / 0001377) 

# Training

In this assignment we will train a basic convolutional neural network to predict the correct imaging series protocol on prostate MRI. At minumum you must include one of the following modern CNN architecture motifs techniques covered in the tutorial:

* residual function with bottleneck operation
* Inception module

You are also **encouraged** to try different permuations and customizations to achieve optimal validation accuracy.

### Define the model

In [0]:
# --- Define model
conv = lambda x, filters,kernel_size : layers.Conv2D(
    filters=filters, 
    kernel_size=kernel_size, 
    padding='same')(x)

proj = lambda filters, x : layers.Conv2D(
    filters=filters, 
    strides=1, 
    kernel_size=(1, 1),
    padding='same')(x)

norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.LeakyReLU()(x)
pool = lambda x : layers.MaxPool2D(pool_size=(3, 3), strides=1, padding='same')(x)

# --- Define 1x1, 3x3 and 5x5 convs
conv1 = lambda filters, x : relu(norm(conv(x, filters, kernel_size=1)))
conv3 = lambda filters, x : relu(norm(conv(x, filters, kernel_size=3)))
conv5 = lambda filters, x : relu(norm(conv(x, filters, kernel_size=5)))
mpool = lambda x : relu(norm(pool(x)))

In [0]:
# --- Define first layer operation
l1 = conv3(16, inputs['dat'])

# --- Define four different paths
filters = 4
b1 = proj(filters, l1)

p1 = conv1(filters, l1)
p2 = conv3(filters, b1)
p3 = conv5(filters, b1)
p4 = proj(filters, mpool(l1))

# --- Concatenate
l2 = layers.Concatenate()([p1, p2, p3, p4])

f0 = layers.Flatten()(l2)

In [0]:
logits = {}
logits['class'] = layers.Dense(4, name='class')(f0)

# --- Create model
model = Model(inputs=inputs, outputs=logits)

### Compile the model

In [0]:
# --- Compile model
model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4), 
    loss={'class': losses.SparseCategoricalCrossentropy(from_logits=True)}, 
    metrics={'class': 'sparse_categorical_accuracy'})

### Train the model

In [9]:
model.fit(
    x=gen_train, 
    steps_per_epoch=250, 
    epochs=12,
    validation_data=gen_valid,
    validation_steps=250,
    validation_freq=4)

Epoch 1/12
250/250 [==============================] - 35s 139ms/step - loss: 2.3670 - sparse_categorical_accuracy: 0.8975
Epoch 2/12
250/250 [==============================] - 35s 139ms/step - loss: 1.0436 - sparse_categorical_accuracy: 0.9522
Epoch 3/12
250/250 [==============================] - 35s 139ms/step - loss: 0.7434 - sparse_categorical_accuracy: 0.9608
Epoch 4/12
250/250 [==============================] - 62s 249ms/step - loss: 0.4773 - sparse_categorical_accuracy: 0.9655 - val_loss: 0.3653 - val_sparse_categorical_accuracy: 0.9778
Epoch 5/12
250/250 [==============================] - 35s 139ms/step - loss: 0.4746 - sparse_categorical_accuracy: 0.9707
Epoch 6/12
250/250 [==============================] - 35s 138ms/step - loss: 0.5006 - sparse_categorical_accuracy: 0.9700
Epoch 7/12
250/250 [==============================] - 34s 138ms/step - loss: 0.3707 - sparse_categorical_accuracy: 0.9765
Epoch 8/12
250/250 [==============================] - 62s 246ms/step - loss: 0.4564 -

# Evaluation

Based on the tutorial discussion, use the following cells to check your algorithm performance. Consider loading a saved model and running prediction using `model.predict(...)` on the data aggregated via a test generator.

**Important**: In this assignment, you must obtain >90% performance accuracy to recieve full credit. Accuracy is determined on a patient by patient (volume by volume) basis, so please *aggregate* results per volume while calculating your performance accuracy here. One common approach is to take the mean prediction across the volume for final prediction; however many altneratives exist. If you determine a better method to calculate accuracy, feel free to implement here.

In [0]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)

**Note**: this cell is used only to check for model performance prior to submission. It will not be graded. Once submitted, your model will be benchmarked against the (same) validation cohort to determine final algorithm performance and grade. If your evaluation code above is correct the algorithm accuracy should match and you can confident that you will recieve full credit for the assignment. Once you are satisfied with your model, proceed to submission of your assignment below.

### Results

When ready, create a `*.csv` file with your compiled **validation** cohort statistics. There is no need to submit training performance accuracy. As in the tutorial, ensure that there are at least three columns in the `*.csv` file:

* true (ground-truth)
* pred (prediction)
* corr (correction prediction, True or False)

In [11]:

trues = []
preds = []

for x, y in test_valid:
    
    # --- Predict
    logits = model.predict(x['dat'][0])

    if type(logits) is dict:
        logits = logits['class']

    # --- Argmax
    pred = np.argmax(logits, axis=1)

    trues.append(y['class'][0, 0])
    preds.append(int(np.round(pred.mean())))

trues = np.array(trues)
preds = np.array(preds)

[ 2020-04-27 18:21:00 ] [====================] 100.000% : Iterating | 000208    

In [13]:
# --- Create *.csv
df = pd.DataFrame(index=np.arange(preds.size))

# --- Define columns
df['true'] = trues
df['pred'] = preds
df['corr'] = df['true'] == df['pred']

# --- Print accuracy
print(df['corr'].mean())           

1.0


In [0]:
# --- Serialize *.csv
fname = '{}/models/series_id/results.csv'.format(MOUNT_ROOT)
os.makedirs(os.path.dirname(fname), exist_ok=True)
df.to_csv(fname)

# Submission

Use the following line to save your model for submission (in Google Colab this should save your model file into your personal Google Drive):

In [0]:
# --- Serialize a model
fname = '{}/models/series_id/final.hdf5'.format(MOUNT_ROOT)
os.makedirs(os.path.dirname(fname), exist_ok=True)
model.save(fname)

### Canvas

Once you have completed this assignment, download the necessary files from Google Colab and your Google Drive. You will then need to submit the following items:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv`
* final (trained) model: `[UCInetID]_model.hdf5`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadshhet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.